# STZ Flow Curve Fitting

**Shear Transformation Zone (STZ) model — Steady-state flow curve**

## Learning Objectives

- Understand the STZ theory: effective temperature chi, activation barriers, and plastic flow
- Fit steady-state flow curves using the analytical arctanh formula
- Visualize C(sigma) and T(sigma) kernel functions
- Perform Bayesian inference with NUTS and evaluate convergence
- Calibrate parameters for use in subsequent STZ notebooks

## Prerequisites

- Basic familiarity with rheological flow curves sigma(gamma_dot)
- Understanding of NLSQ fitting (see `01-basic-maxwell.ipynb`)
- Understanding of Bayesian inference basics (see `05-bayesian-basics.ipynb`)

## Estimated Runtime

- Fast demo (1 chain): ~1-2 min
- Full run (4 chains): ~4-6 min

## 1. Setup

In [ ]:
# Colab setup
import sys

IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"
    print("RheoJAX installed successfully.")

In [ ]:
%matplotlib inline
import os
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.stz import STZConventional

jax, jnp = safe_import_jax()
verify_float64()

warnings.filterwarnings("ignore", category=FutureWarning)
print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")

## 2. Theory: Shear Transformation Zones

The STZ model (Falk & Langer 1998, Langer 2008) describes plastic flow in **amorphous solids** — metallic glasses, colloidal glasses, dense emulsions — via localized rearrangements called *Shear Transformation Zones*.

### Key concepts

**Effective temperature** $\chi$: A configurational entropy measure of structural disorder. Unlike thermal temperature, $\chi$ is driven by mechanical work and controls the density of STZs:

$$\Lambda = \exp(-e_z / \chi)$$

where $e_z$ is the STZ formation energy and $\Lambda$ is the equilibrium STZ density.

**Rate factor** $\mathcal{C}(\sigma) = \cosh(\sigma / \sigma_y)$ — total transition rate (forward + backward).

**Transition bias** $\mathcal{T}(\sigma) = \tanh(\sigma / \sigma_y)$ — net directional bias of transformations.

**Plastic strain rate:**

$$\dot{\gamma}_{\text{pl}} = \frac{2 \epsilon_0}{\tau_0} \, \Lambda \, \exp(-1/\chi) \, \mathcal{C}(\sigma) \, \mathcal{T}(\sigma)$$

### Steady-state flow curve

At steady state, $\chi \to \chi_{\infty}$ and the combined activation factor simplifies. Setting $\dot{\gamma} = \dot{\gamma}_{\text{pl}}$ and inverting yields the **arctanh formula**:

$$\sigma = \sigma_y \cdot \text{arctanh}\!\left(\frac{\dot{\gamma} \, \tau_0}{\exp\!\left[-(1 + e_z)/\chi_{\infty}\right]}\right)$$

This is **analytical** (no ODE integration), making flow curve fitting fast.

### Parameters for steady state

| Parameter | Symbol | Physical meaning |
|-----------|--------|------------------|
| `sigma_y` | $\sigma_y$ | Yield stress — activation barrier scale |
| `chi_inf` | $\chi_{\infty}$ | Steady-state effective temperature |
| `tau0` | $\tau_0$ | Molecular attempt time |
| `ez` | $e_z$ | STZ formation energy |

**Note:** G0, epsilon0, c0, and tau_beta only affect *transient* response and are **not identifiable** from steady-state data alone.

### Synthetic Data Strategy

We generate **synthetic flow curve data** from the STZ arctanh formula with known parameters, then add 3% log-normal noise. This ensures the model-data compatibility is exact, letting us focus on the fitting workflow and parameter identifiability.

The STZ arctanh formula was derived for amorphous solids where the transition from thermally-activated creep to driven flow is sharp. Real yield-stress materials (emulsions, colloidal gels) often show smoother Herschel-Bulkley behavior that the arctanh shape cannot capture. Notebooks 03-05 use real experimental data with other protocols (relaxation, creep, SAOS) where STZ performs well via ODE integration.

## 3. Generate Synthetic Data

In [ ]:
from stz_tutorial_utils import generate_synthetic_flow_curve

gamma_dot, stress, true_params = generate_synthetic_flow_curve()

print(f"Data points: {len(gamma_dot)}")
print(f"Shear rate range: {gamma_dot.min():.4f} - {gamma_dot.max():.1f} 1/s")
print(f"Stress range: {stress.min():.4f} - {stress.max():.1f} Pa")
print("\nTrue parameters:")
for k, v in true_params.items():
    print(f"  {k:8s} = {v:.4g}")

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
ax.loglog(gamma_dot, stress, "ko", markersize=6, label="Synthetic data (3% noise)")
ax.set_xlabel("Shear rate [1/s]")
ax.set_ylabel("Stress [Pa]")
ax.set_title("STZ Synthetic Flow Curve")
ax.grid(True, alpha=0.3, which="both")
ax.legend()
plt.tight_layout()
display(fig)
plt.close(fig)

The arctanh shape shows: linear Newtonian regime at low shear rates ($\sigma \propto \dot{\gamma}$), then a sharp upturn as the argument approaches unity. This transition is controlled by $\tau_0$ and $\chi_{\infty}$, while the overall stress scale is set by $\sigma_y$.

## 4. NLSQ Fitting

In [ ]:
import time
from stz_tutorial_utils import compute_fit_quality

model = STZConventional(variant="standard")

# Non-identifiable params: constrain to tight bounds (they don't affect steady state
# but large values cause the optimizer to stall via xtol)
model.parameters.set_bounds("G0", (0.5, 2.0))
model.parameters["G0"].value = 1.0
model.parameters.set_bounds("epsilon0", (0.05, 0.2))
model.parameters["epsilon0"].value = 0.1
model.parameters.set_bounds("c0", (0.5, 2.0))
model.parameters["c0"].value = 1.0
model.parameters.set_bounds("tau_beta", (0.5, 2.0))
model.parameters["tau_beta"].value = 1.0

# Identifiable params — set bounds and initial guesses (offset from true values)
model.parameters.set_bounds("sigma_y", (1.0, 500.0))
model.parameters["sigma_y"].value = 30.0
model.parameters.set_bounds("chi_inf", (0.05, 0.5))
model.parameters["chi_inf"].value = 0.20
model.parameters.set_bounds("tau0", (1e-8, 1e0))
model.parameters["tau0"].value = 1e-3
model.parameters.set_bounds("ez", (0.1, 5.0))
model.parameters["ez"].value = 1.5

t0 = time.time()
model.fit(gamma_dot, stress, test_mode="steady_shear", use_log_residuals=True, method='scipy')
t_nlsq = time.time() - t0

stress_at_data = model.predict(gamma_dot)
quality = compute_fit_quality(stress, stress_at_data)

print(f"NLSQ fit time: {t_nlsq:.2f} s")
print(f"R-squared: {quality['r_squared']:.6f}")
print(f"RMSE: {quality['rmse']:.4f} Pa")
print("\nFitted vs true parameters (steady-state):")
for name in ["sigma_y", "chi_inf", "tau0", "ez"]:
    fitted = model.parameters.get_value(name)
    true = true_params[name]
    print(f"  {name:8s} = {fitted:.4g}  (true: {true:.4g})")

### 4.1 Fit Quality

In [ ]:
gamma_dot_fine = np.logspace(
    np.log10(gamma_dot.min()) - 0.5,
    np.log10(gamma_dot.max()) + 0.2,
    200,
)
stress_pred = model.predict(gamma_dot_fine)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Left: flow curve
ax1.loglog(gamma_dot, stress, "ko", markersize=6, label="Data")
ax1.loglog(gamma_dot_fine, stress_pred, "-", lw=2, color="C0", label="STZ fit")
ax1.set_xlabel("Shear rate [1/s]")
ax1.set_ylabel("Stress [Pa]")
ax1.set_title("STZ Flow Curve Fit")
ax1.legend()
ax1.grid(True, alpha=0.3, which="both")

# Right: residuals
stress_at_data = model.predict(gamma_dot)
res = (stress - stress_at_data) / stress * 100
ax2.semilogx(gamma_dot, res, "o-", markersize=5, color="C0")
ax2.axhline(0, color="black", linestyle="--", alpha=0.5)
ax2.set_xlabel("Shear rate [1/s]")
ax2.set_ylabel("Relative residual [%]")
ax2.set_title("Residual Analysis")
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

### 4.2 Physics Insight: C(sigma) and T(sigma) Kernels

The rate factor $\mathcal{C}$ and transition bias $\mathcal{T}$ are the two fundamental kernels that control STZ dynamics. Let's visualize them with the fitted yield stress.

In [ ]:
from rheojax.models.stz._kernels import rate_factor_C, transition_T

sigma_y_fit = model.parameters.get_value("sigma_y")
sigma_range = np.linspace(0, 3 * sigma_y_fit, 200)

C_vals = np.array([float(rate_factor_C(s, sigma_y_fit)) for s in sigma_range])
T_vals = np.array([float(transition_T(s, sigma_y_fit)) for s in sigma_range])

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(sigma_range / sigma_y_fit, C_vals, "-", lw=2.5, color="C1")
ax1.axvline(1.0, color="gray", linestyle=":", alpha=0.7, label="$\\sigma = \\sigma_y$")
ax1.set_xlabel("$\\sigma / \\sigma_y$")
ax1.set_ylabel("$\\mathcal{C}(\\sigma)$")
ax1.set_title("Rate Factor: $\\mathcal{C} = \\cosh(\\sigma/\\sigma_y)$")
ax1.set_yscale("log")
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(sigma_range / sigma_y_fit, T_vals, "-", lw=2.5, color="C2")
ax2.axvline(1.0, color="gray", linestyle=":", alpha=0.7, label="$\\sigma = \\sigma_y$")
ax2.axhline(1.0, color="gray", linestyle="--", alpha=0.3)
ax2.set_xlabel("$\\sigma / \\sigma_y$")
ax2.set_ylabel("$\\mathcal{T}(\\sigma)$")
ax2.set_title("Transition Bias: $\\mathcal{T} = \\tanh(\\sigma/\\sigma_y)$")
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

- **$\mathcal{C}(\sigma)$** grows exponentially above $\sigma_y$, accelerating total rearrangement rate.
- **$\mathcal{T}(\sigma)$** saturates at 1.0 for $\sigma \gg \sigma_y$, meaning all transformations are forward-biased.
- Below $\sigma_y$: forward and backward transformations nearly cancel ($\mathcal{T} \approx 0$), so no net plastic flow.

## 5. Bayesian Inference with NUTS

### 5.1 Run NUTS

In [ ]:
# Warm-start values from NLSQ
initial_values = {
    name: model.parameters.get_value(name)
    for name in model.parameters.keys()
}
print("Warm-start values:")
for k, v in initial_values.items():
    print(f"  {k}: {v:.4g}")

# --- Fast demo config ---
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1
# NUM_WARMUP = 1000; NUM_SAMPLES = 2000; NUM_CHAINS = 4  # production

t0 = time.time()
result = model.fit_bayesian(
    gamma_dot,
    stress,
    test_mode="steady_shear",
    num_warmup=NUM_WARMUP,
    num_samples=NUM_SAMPLES,
    num_chains=NUM_CHAINS,
    initial_values=initial_values,
    seed=42,
)
t_bayes = time.time() - t0
print(f"\nBayesian inference time: {t_bayes:.1f} s")

### 5.2 Convergence Diagnostics

In [ ]:
from stz_tutorial_utils import print_convergence_summary

param_names = ["sigma_y", "chi_inf", "tau0", "ez"]
print_convergence_summary(result, param_names)

### 5.3 ArviZ Diagnostic Plots

In [ ]:
idata = result.to_inference_data()

# Trace plot
axes = az.plot_trace(idata, var_names=param_names, figsize=(12, 8))
fig = axes.ravel()[0].figure
fig.suptitle("Trace Plots", fontsize=14, y=1.02)
plt.tight_layout()
display(fig)
plt.close(fig)

In [ ]:
# Pair plot (parameter correlations)
axes = az.plot_pair(
    idata,
    var_names=param_names,
    kind="scatter",
    divergences=True,
    figsize=(10, 10),
)
fig = axes.ravel()[0].figure
fig.suptitle("Parameter Correlations", fontsize=14, y=1.02)
plt.tight_layout()
display(fig)
plt.close(fig)

In [ ]:
# Forest plot (credible intervals)
axes = az.plot_forest(
    idata,
    var_names=param_names,
    combined=True,
    hdi_prob=0.95,
    figsize=(10, 4),
)
fig = axes.ravel()[0].figure
plt.tight_layout()
display(fig)
plt.close(fig)

### 5.4 Posterior Predictive Check

In [ ]:
posterior = result.posterior_samples
n_draws = min(200, len(list(posterior.values())[0]))

gamma_dot_pred = np.logspace(
    np.log10(gamma_dot.min()) - 0.3,
    np.log10(gamma_dot.max()) + 0.2,
    100,
)
gamma_dot_pred_jax = jnp.asarray(gamma_dot_pred, dtype=jnp.float64)

# Draw posterior predictive samples
pred_samples = []
for i in range(n_draws):
    pred_i = STZConventional._predict_steady_shear_jit(
        gamma_dot_pred_jax,
        posterior["sigma_y"][i],
        posterior["chi_inf"][i],
        posterior["tau0"][i],
        posterior["ez"][i],
    )
    pred_samples.append(np.array(pred_i))

pred_samples = np.array(pred_samples)
pred_median = np.median(pred_samples, axis=0)
pred_lo = np.percentile(pred_samples, 2.5, axis=0)
pred_hi = np.percentile(pred_samples, 97.5, axis=0)

fig, ax = plt.subplots(figsize=(9, 6))
ax.fill_between(
    gamma_dot_pred, pred_lo, pred_hi, alpha=0.3, color="C0", label="95% CI"
)
ax.loglog(gamma_dot_pred, pred_median, "-", lw=2, color="C0", label="Posterior median")
ax.loglog(gamma_dot, stress, "ko", markersize=6, label="Data")
ax.set_xlabel("Shear rate [1/s]")
ax.set_ylabel("Stress [Pa]")
ax.set_title("Posterior Predictive Check")
ax.legend()
ax.grid(True, alpha=0.3, which="both")
plt.tight_layout()
display(fig)
plt.close(fig)

### 5.5 Parameter Summary

In [ ]:
from stz_tutorial_utils import print_parameter_comparison

print_parameter_comparison(model, posterior, param_names)

## 6. Common Failure Modes

| Issue | Symptom | Fix |
|-------|---------|-----|
| arctanh argument > 1 | NaN in predictions | Reduce tau0 or increase chi_inf so activation term is large enough |
| sigma_y at lower bound | Yield stress too low | Widen lower bound or check data units (Pa vs kPa) |
| chi_inf-tau0 correlation | Banana-shaped posterior | Expected — both control activation; combine with transient data |
| Poor low-rate fit | Residuals large at low gamma_dot | Enable `use_log_residuals=True` (default) to weight all decades equally |

## 7. Save Results

Save the calibrated parameters for use in Notebooks 02 and 06 (which generate synthetic startup and LAOS data from these values).

In [ ]:
from stz_tutorial_utils import save_stz_results

output_dir = os.path.join("..", "outputs", "stz", "flow_curve")
save_stz_results(model, result, output_dir, "flow_curve")

## Key Takeaways

1. **STZ captures yield stress via activation barriers** — the arctanh formula directly encodes the competition between thermal activation and mechanical driving
2. **Only 4 parameters from steady state** — G0, epsilon0, c0, tau_beta require transient data (startup, relaxation, creep)
3. **chi_inf encodes structural disorder** — higher chi_inf means more disordered (more STZs available), lower activation barrier
4. **tau0 and chi_inf are correlated** — they jointly control the activation prefactor; Bayesian inference quantifies this correlation
5. **C(sigma) and T(sigma) provide physical intuition** — rate factor vs directional bias decompose plastic flow into distinct mechanisms

## Next Steps

- **Notebook 02**: Use these calibrated parameters to generate startup shear data and observe stress overshoot
- **Notebook 03**: Fit stress relaxation data from laponite clay (aging behavior)
- **Notebook 05**: Fit SAOS data using the Maxwell-like linear approximation